# 3.4 功能预测任务

## 数据准备

获得蛋白质序列及其对应的功能描述是生物信息学中的一个重要任务，通常涉及从公共数据库中检索数据或通过实验方法确定。以下是几种常用的方法和资源，帮助你获取蛋白质序列和功能描述：

### 1. **使用公共数据库**

#### a. **UniProt (Universal Protein Resource)**

- **网址**：[UniProt](https://www.uniprot.org/)
- **特点**：UniProt 是一个综合性的蛋白质数据库，提供了丰富的注释信息，包括蛋白质序列、结构、功能、亚细胞定位等。
- **使用方法**：
  - 在搜索栏中输入蛋白质名称、基因名称或序列 ID。
  - 浏览结果页面以查看详细的注释信息，包括功能描述、GO（Gene Ontology）术语、文献引用等。


<img src="img/function.png" width="500px" />

<img src="img/sequence.png" width="500px" />

#### b. **NCBI (National Center for Biotechnology Information)**

- **网址**：[NCBI](https://www.ncbi.nlm.nih.gov/)
- **特点**：NCBI 提供多个相关数据库，如 GenBank、RefSeq 和 Protein 数据库，涵盖广泛的生物物种和蛋白质信息。
- **使用方法**：
  - 使用 NCBI 的搜索工具 Entrez 或 BLAST 搜索蛋白质序列或功能描述。
  - 访问特定的蛋白质条目以获取详细信息，包括序列、功能、参考文献等。

#### c. **PDB (Protein Data Bank)**

- **网址**：[PDB](https://www.rcsb.org/)
- **特点**：PDB 主要包含蛋白质的三维结构信息，但也提供相关的功能描述和文献引用。
- **使用方法**：
  - 使用 PDB 的搜索功能查找特定蛋白质的结构信息。
  - 查看每个条目的详细页面以获取功能描述和其他相关信息。

#### d. **Ensembl**

- **网址**：[Ensembl](https://www.ensembl.org/)
- **特点**：Ensembl 提供基因组浏览器和注释信息，涵盖了多种物种的基因和蛋白质数据。
- **使用方法**：
  - 使用 Ensembl 的搜索功能查找特定蛋白质或基因。
  - 浏览条目页面以获取详细的注释信息，包括功能描述、GO 术语等。

### 2. **通过生物信息学工具**

#### a. **BLAST (Basic Local Alignment Search Tool)**

- **网址**：[BLAST](https://blast.ncbi.nlm.nih.gov/Blast.cgi)
- **特点**：BLAST 是一种常用的比对工具，可以帮助你根据已知的蛋白质序列找到相似的序列，并获取其功能描述。
- **使用方法**：
  - 输入你的蛋白质序列。
  - 选择适当的数据库（如 NR、Swiss-Prot 等）进行比对。
  - 分析比对结果，查看相似序列的功能描述。

#### b. **InterProScan**

- **网址**：[InterPro](https://www.ebi.ac.uk/interpro/)
- **特点**：InterProScan 是一种用于识别蛋白质家族、结构域和重要位点的工具，可以提供详细的注释信息。
- **使用方法**：
  - 输入你的蛋白质序列。
  - 运行 InterProScan 分析，获取功能描述、结构域信息等。

### 3. **通过文献和出版物**

#### a. **PubMed**

- **网址**：[PubMed](https://pubmed.ncbi.nlm.nih.gov/)
- **特点**：PubMed 是一个广泛使用的生物医学文献数据库，提供了大量关于蛋白质功能的研究论文。
- **使用方法**：
  - 使用关键词搜索与特定蛋白质相关的研究论文。
  - 阅读论文以获取详细的实验数据和功能描述。

#### b. **Google Scholar**

- **网址**：[Google Scholar](https://scholar.google.com/)
- **特点**：Google Scholar 是一个学术搜索引擎，涵盖广泛的科学文献。
- **使用方法**：
  - 使用关键词搜索与特定蛋白质相关的研究论文。
  - 阅读论文以获取详细的实验数据和功能描述。

## 整理好的数据

https://huggingface.co/datasets/PharMolix/MutaDescribe

<img src="img/dataset.png" width="500px" />

https://huggingface.co/datasets/jonghyunlee/UniProt_function_text_descriptions?row=2

In [1]:
import subprocess
import os
# 设置环境变量, autodl一般区域
result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

"""
import os

# 设置环境变量, autodl专区 其他idc
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

# 打印环境变量以确认设置成功
print(os.environ.get('HF_ENDPOINT'))
"""

"\nimport os\n\n# 设置环境变量, autodl专区 其他idc\nos.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'\n\n# 打印环境变量以确认设置成功\nprint(os.environ.get('HF_ENDPOINT'))\n"

In [2]:
from transformers import AutoTokenizer, AutoModel
from tokenizers import Tokenizer
from transformers import GPT2LMHeadModel, AutoConfig,GPT2Tokenizer
from transformers import AutoModelForSeq2SeqLM 
from transformers import DataCollatorWithPadding

In [3]:
#set tokenizer,dna protein 
tokenizer = GPT2Tokenizer.from_pretrained("dnagpt/gene_eng_gpt2_v0")
tokenizer.pad_token = tokenizer.eos_token

In [4]:
from datasets import load_dataset
# 1. load ~11k samples from promoters prediction dataset
dataset = load_dataset("jonghyunlee/UniProt_function_text_descriptions")['train'].select(range(5000)).train_test_split(test_size=0.05)

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['entry', 'entry_name', 'protein_name', 'sequence', 'function'],
        num_rows: 4750
    })
    test: Dataset({
        features: ['entry', 'entry_name', 'protein_name', 'sequence', 'function'],
        num_rows: 250
    })
})

In [6]:
dataset["train"][0]

{'entry': 'A1TFU9',
 'entry_name': 'HPXO_MYCVP',
 'protein_name': 'FAD-dependent urate hydroxylase (EC 1.14.13.113) (Flavoprotein urate hydroxylase)',
 'sequence': 'MKVVIVGAGMGGMSAAIALRQIGIDTVVYERVTENKPVGAAISVWSNGVKCLNYLGLQEETAELGGKVETMSYVDGHTGDTMCRFSMHPLIEQVGQRPYPIARAELQLMLMKAYGIDDINFGMKMVGVENDTAGSAAKATFADGTTVSADVIIGADGAGSITREYVLGGPVSRRYAGYVNYNGLVSTDDAIGPATEWTTYVGDGKRVSVMPVSDDRFYFFFDVVEPQGSPYEEGRVREVLRAHFAGWTPGVQTLIDTLDPLATNRVEILDLDPFHTWVKGRVAVLGDAAHNTTPDIGQGGCSAMEDAIALQWAFKDHPDDVHAALAAYQSARTERAADLVLRARKRCDVTHAKDPQVTSRWYDELRNEDGTNIIRGIVGNIVGGPLTPVTAATEG',
 'function': 'Catalyzes the hydroxylation of urate to 5-hydroxyisourate (HIU). Is likely to be involved in the urate degradation pathway to allantoin. Prefers NADH over NADPH as the electron donor. '}

In [10]:
token_len_list = []
for item in dataset["test"].select(range(200)):
    inputs = tokenizer.tokenize(item["sequence"])
    token_len_list.append( len(inputs) )

mean_len = sum(token_len_list)/len(token_len_list)
min_len  = min(token_len_list)
max_len = max(token_len_list)

print("dna datasets ", "mean token lenght", mean_len, "min token length", min_len, "max token length", max_len)

dna datasets  mean token lenght 269.515 min token length 24 max token length 4577


In [11]:
token_len_list = []
for item in dataset["test"].select(range(50)):
    inputs = tokenizer.tokenize(item["function"])
    token_len_list.append( len(inputs) )

mean_len = sum(token_len_list)/len(token_len_list)
min_len  = min(token_len_list)
max_len = max(token_len_list)

print("dna datasets ", "mean token lenght", mean_len, "min token length", min_len, "max token length", max_len)

dna datasets  mean token lenght 271.02 min token length 23 max token length 1934


In [6]:
max_length = 128

def preprocess_function(examples):
    # 直接从 examples 中提取字段
    inputs = examples["sequence"]  # 获取所有样本的 "sequence"
    targets = examples["function"]  # 获取所有样本的 "function"

    # 对数据进行编码
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs


# 应用分词
tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names,)


Map:   0%|          | 0/4750 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

In [10]:
# 查看处理后的数据，使用正确的拆分（例如 'train'）
print(tokenized_datasets['train'][0])  # 查看 'train' 数据集中的第一个样本
print(tokenized_datasets["test"][0])# 查看 'test' 数据集中的第一个样本


{'input_ids': [9206, 609, 532, 2065, 487, 1241, 50, 1785, 4070, 827, 28817, 3840, 1105, 8309, 2993, 3449, 47036, 22588, 5215, 636, 4189, 12265, 3721, 7075, 69183, 3040, 814, 1209, 1910, 217, 474, 13943, 15033, 535, 558, 51164, 12333, 56886, 1174, 338, 20934, 9865, 46, 1131, 3021, 336, 11005, 20318, 748, 396, 46, 38, 46, 54, 12036, 482, 4807, 284, 13333, 87969, 1482, 618, 371, 46, 49, 29703, 46, 5669, 55496, 40, 2682, 2186, 84535, 471, 12020, 280, 1751, 46, 545, 3968, 1660, 354, 1309, 84775, 328, 3802, 52, 46, 33718, 797, 46, 39, 487, 965, 16953, 790, 8503, 53823, 365, 39878, 41235, 17957, 25823, 785, 967, 1371, 543, 8660, 1510, 308, 46, 46, 35663, 3804, 4662, 15100, 8524, 2378, 254, 2399, 38462, 1700, 3223, 1296, 478, 1972, 809, 251], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [16]:
from transformers import Trainer, TrainingArguments, GPT2LMHeadModel
from transformers import DataCollatorForSeq2Seq


# 加载预训练的 GPT-2 模型
model = GPT2LMHeadModel.from_pretrained("dnagpt/gene_eng_gpt2_v0")

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# 训练参数
training_args = TrainingArguments(
    output_dir="gpt2_protein_function",  # 输出目录
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    save_steps=500,
    save_total_limit=2,
    logging_steps=500,
)

# 初始化 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],  # 传递 'train' 拆分数据
    eval_dataset=tokenized_datasets['test'],   # 传递 'test' 拆分数据
    data_collator=data_collator,
)

# 训练模型
trainer.train()


model.safetensors:  69%|######9   | 430M/620M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

/root/miniconda3/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


[2025-01-05 16:06:01,794] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/root/miniconda3/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/root/miniconda3/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3'
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::ostream::tellp()@GLIBCXX_3.4'
/root/minicond

Epoch,Training Loss,Validation Loss
1,5.123600,4.953184
2,4.958600,4.907465
3,4.893500,4.876421
4,4.801000,4.860046
5,4.709600,4.876781


TrainOutput(global_step=2970, training_loss=4.859284495344066, metrics={'train_runtime': 183.1426, 'train_samples_per_second': 129.68, 'train_steps_per_second': 16.217, 'total_flos': 1551421440000000.0, 'train_loss': 4.859284495344066, 'epoch': 5.0})

In [ ]:
# 保存训练好的模型和 tokenizer
model.save_pretrained(run_path)
tokenizer.save_pretrained(run_path)


In [19]:
import math
# 评估模型在验证集上的表现
eval_results = trainer.evaluate()
print(eval_results)
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

{'eval_loss': 4.8767805099487305, 'eval_runtime': 0.489, 'eval_samples_per_second': 511.252, 'eval_steps_per_second': 65.44, 'epoch': 5.0}
Perplexity: 131.21
